# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [28]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [29]:
from cassandra.cluster import Cluster
cluster = Cluster()
session = cluster.connect()

#### Create Keyspace

In [30]:
session.execute("CREATE KEYSPACE IF NOT EXISTS udacity WITH replication = {'class':'SimpleStrategy','replication_factor':1}")

#### Set Keyspace

In [31]:
session.set_keyspace('udacity')

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'





**artist_song_session Table for first query**

In the first query, the output columns should be Artist Name, Song Title and Song's Length. The filters will be on sessionId and itemInSession. Hence we need all of these columns in the table. Our Table structure will be as follows:

| Column        |  Type    |
|---------------|----------|
| sessionId     |  int     |
| itemInSession |  int     |
| artist        |  text    |
| song          |  text    |
| length        |  decimal |

The filtering criteria is on sessionId and itemInSession columns. 
As per data, sessionId has duplicate values. While the combination of sessionId and itemInSession is unique. Hence we can design them as primary key with itemInSession as Partition Key. 

Based on the information we are storing in the table, we are naming the table as ***artist_song_session***

In [32]:
query1 = """ CREATE TABLE IF NOT EXISTS artist_song_session 
             (sessionId int,itemInSession int,artist text,song text,length decimal,PRIMARY KEY(sessionId,itemInSession)) """
session.execute(query1)

In [33]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for line in csvreader:
        query = "INSERT INTO artist_song_session(sessionId,itemInSession,artist,song,length)"
        query = query + "VALUES(%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

In [34]:
rows = session.execute("select artist,song,length from artist_song_session where sessionId = 338 and itemInSession = 4")
for row in rows:
    print(row.artist,row.song,row.length)

Faithless Music Matters (Mark Knight Dub) 495.3073


### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

**songs_playlist_session Table for second query**

In the second query, the output columns should be artist, song and user, where song should be sorted by itemInSession and user should contain the firstname and lastname. The filters will be on userId and sessionId. Hence we need 6 columns in the table (userId, sessionId, itemInSession, artist, song, user). Here user will be a concatenated field of firstName and lastName.

| Column        |  Type    |
|---------------|----------|
| userId        |  int     |
| sessionId     |  int     |
| itemInSession |  int     |
| artist        |  text    |
| song          |  text    |
| user          |  text    |

The filtering criteria is on userId and sessionId columns. The song should be sorted by itemInSession. Also per data, combination of userId, sessionId and itemInSession will give unique records. Hence we will design userId and sessonId as composite Primary Key while the itemInSession column will be used as Partition Key.

Based on the information we are storing in the table, we are naming the table as ***songs_playlist_session***

In [35]:
query2 = """CREATE TABLE IF NOT EXISTS songs_playlist_session
            (userId int,sessionId int,itemInSession int,artist text,song text,user text, PRIMARY KEY((userId,sessionId),itemInSession))"""

session.execute(query2)


In [36]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO songs_playlist_session (userId,sessionId,itemInSession,artist,song,user)"
        query = query + "VALUES(%s,%s,%s,%s,%s,%s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1]+' '+line[4]))

In [37]:
rows = session.execute("select artist,song,user from songs_playlist_session where userId = 10 and sessionId = 182")
for row in rows:
    print(row.artist,row.song,row.user)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


**user_song Table for third query**

In the third query, the output columns should be te user names based on the filter on song. 2 different user can have same firstName and lastName. Hence to make the records unique, we wll need userId as well. So following will be the table structure:

| Column   |  Type  |
|----------|--------|
| song     |  text  |
| userid   |  int   |
| user     |  text  |

Filters will be on Songs. And userId is needed to find unique records. Hence we will design them as primary key with userId will be the Partition Key. 

Based on the information we are storing in the table, we are naming the table as ***user_song***

In [38]:
query3 = "CREATE TABLE IF NOT EXISTS user_song(song text,userid int, user text, PRIMARY KEY(song,userid)) " 
session.execute(query3)


In [39]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_song (song,userid,user)"
        query = query + "VALUES(%s,%s,%s)"
        session.execute(query, ( line[9], int(line[10]), line[1]+' '+line[4]))

In [40]:
rows = session.execute("select user from user_song where song = 'All Hands Against His Own' ")
for row in rows:
    print(row.user)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [41]:
session.execute("DROP TABLE artist_song_session")
session.execute("DROP TABLE songs_playlist_session")
session.execute("DROP TABLE user_song")

### Close the session and cluster connection¶

In [42]:
session.shutdown()
cluster.shutdown()